In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# GPT-2 모델 및 토크나이저 로딩
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 모델의 토크나이저 최대 입력 길이 설정
max_length = model.config.n_positions

def predict_time_of_day(input_text):
    # 입력 텍스트를 토큰화
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length)

    # 모델을 통해 예측
    with torch.no_grad():
        logits = model(input_ids).logits[:, -1, :]
    
    # 확률 분포에서 가장 높은 값을 가지는 인덱스 찾기
    predicted_time_index = torch.argmax(logits, dim=-1).item()

    return predicted_time_index

In [5]:
# 예시 입력 텍스트
input_text = input("검색어 입력하세요:")

# 모델을 통해 예측된 시간대 출력
predicted_time = predict_time_of_day(input_text)
print(f"입력 텍스트에 대한 예측된 시간대: {predicted_time}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


입력 텍스트에 대한 예측된 시간대: 13


In [1]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

# Example dataset (replace this with your actual dataset)
train_dataset = [
    {"text": "아침에 일어나서 커피를 마셨다.", "label": 0},  # 0: 아침
    {"text": "오후에 회의를 진행했다.", "label": 1},  # 1: 오후
    {"text": "저녁 식사 후 산책을 즐겼다.", "label": 2}  # 2: 저녁
    # ... add more data
]

# Tokenize the dataset
tokenized_datasets = tokenizer(train_dataset, padding=True, truncation=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_sequence_classification",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./gpt2_finetuned_sequence_classification")

# Now you can use the fine-tuned model for prediction


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.